In [4]:
import torch
import numpy as np
from torch import cuda
from torchvision import datasets, transforms
import os

In [5]:
tr =  transforms.Compose([transforms.ToTensor(),transforms.Lambda(lambda x: torch.flatten(x))])
training_data = datasets.FashionMNIST(".",download=True,train=True, transform = tr)
test_data = datasets.FashionMNIST(".",download=True,train=False, transform = tr)

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
cuda.is_available()

True

In [6]:
type(training_data[1][0])

torch.Tensor

In [6]:
training_data[1][0].size()

torch.Size([784])

In [7]:

model_TD1 = torch.nn.Sequential(
    torch.nn.Linear(784, 1),
).cuda()
#print(type(model_TD1(training_data[1][0]).cuda()), type(training_data[1][1]))
#loss_fn(model_TD1(training_data[1][0]), )

In [8]:
optimizer = torch.optim.Adam(model_TD1.parameters())
loss_fn = torch.nn.MSELoss()

In [9]:
from tqdm.notebook import tqdm
from time import sleep
from tqdm import trange, tnrange

In [10]:
def trainer(dataset,model,loss_fn,epoch=10,batch_size=1,rate=1e-4):
  optimizer = torch.optim.Adam(model.parameters(), lr= rate)
  ##loss_fn = loss_fn
  for i in range(epoch):
    dataload = torch.utils.data.DataLoader(dataset, batch_size = batch_size)
    for batch, (x,y) in enumerate(dataload):
    #for (x,y) in tqdm(dataload):
      # Compute prediction error
      y_pred = model(x.to(device))
      loss =loss_fn(y_pred.float(), y.float().to(device))
      # Backpropagation
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
  return model


In [11]:
mdl = trainer(training_data, model_TD1, torch.nn.MSELoss(), epoch = 2)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [12]:
def success_rate(model,test_data):
  S = 0
  n = len(test_data)
  #k = len(model(test_data[1][0]))
  for i in range(n):
      prediction = model(test_data[i][0].to(device))
      if np.abs((float(prediction- test_data[i][1])))<0.5:
        S += 1
  return S/n

In [13]:
len(test_data[0])

2

In [14]:
model_TD1(training_data[0][0].to(device))[0]

tensor(9.9599, device='cuda:0', grad_fn=<SelectBackward>)

In [15]:
success_rate(mdl, test_data)

0.3343

In [16]:
model_10 = torch.nn.Sequential(torch.nn.Linear(784, 10), torch.nn.LogSoftmax()).to(device)



In [17]:
def trainer2(dataset,model,loss_fn,epoch=10,batch_size=1,rate=1e-4):
  optimizer = torch.optim.Adam(model.parameters(), lr= rate)
  ##loss_fn = loss_fn
  for i in range(epoch):
    dataload = torch.utils.data.DataLoader(dataset, batch_size = batch_size)
    #for batch, (x,y) in enumerate(dataload):
    for (x,y) in tqdm(dataload):
      # Compute prediction error
      x,y = x.to(device), y.to(device)
      yv_pred = model(x)
      #y_pred = np.argmax([list(yv_pred)])
      #loss = loss_fn(y_pred)
      loss =loss_fn(yv_pred.float(), y)
      # Backpropagation
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      #sleep(0.001)
  return model


In [18]:
mdl2 = trainer2(training_data, model_10, torch.nn.NLLLoss(), epoch = 3)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [19]:
def success_rate2(model,test_data):
  S = 0
  n = len(test_data)
  #k = len(model(test_data[1][0]))
  for i in range(n):
      prediction_vector = model(test_data[i][0].to(device))
      prediction = np.argmax(list(prediction_vector))
      if np.abs((float(prediction- test_data[i][1])))<0.5:
        S += 1
  return S/n

In [20]:
success_rate2(mdl2, test_data)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0.8373

In [21]:
model_reLu_activation = torch.nn.Sequential(torch.nn.Linear(784, 128), torch.nn.ReLU(),torch.nn.Linear(128, 10), torch.nn.LogSoftmax()).to(device)
trainer2(training_data, model_reLu_activation, torch.nn.NLLLoss(), epoch = 10)
success_rate2(model_reLu_activation, test_data)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0.8836

In [22]:
model_sigmoid_activation = torch.nn.Sequential(torch.nn.Linear(784, 128), torch.nn.Sigmoid(),torch.nn.Linear(128, 10) ,torch.nn.LogSoftmax()).to(device)
trainer2(training_data, model_sigmoid_activation, torch.nn.NLLLoss(), epoch = 2)
success_rate2(model_sigmoid_activation, test_data)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0.8481

In [23]:
model_tanh_activation = torch.nn.Sequential(torch.nn.Linear(784, 10), torch.nn.Tanh(), torch.nn.Linear(10, 10),torch.nn.LogSoftmax()).to(device)
trainer2(training_data, model_tanh_activation, torch.nn.NLLLoss(), epoch = 2)
success_rate2(model_tanh_activation, test_data)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0.8332

In [24]:
import cloudpickle as pickle
from google.colab import files

def torch_saver(net,file="temp"):
  with open(file,'wb') as f:
    pickle.dump(net,f)
    files.download(file)


In [25]:
#torch_saver(model_reLu_activation)

In [26]:
import torch
from torch import nn
import torch.nn.functional as F 

class MyNetwork(nn.Module): # a class inheriting from nn.Module

    def __init__(self): 
        super(MyNetwork, self).__init__() # call the constructor of nn.Module
        # you can now define some layers
        self.layer1 = nn.Linear(784,128)
        self.layer2 = nn.Linear(128,10)
        self.layer3 = nn.LogSoftmax()
    def forward(self,x): 
        # how do the layers compute the output ?
        # this function needs to return the output of the net
        # usually by applying the layers in the right order
        ...
        x = x.to(device)
        result = F.relu(self.layer1(x))
        result = self.layer2(result)
        result = self.layer3(result)
        return result

In [27]:
model_cl = MyNetwork().to(device)

In [28]:
trainer2(training_data, model_cl, torch.nn.NLLLoss(), epoch = 3)
success_rate2(model_cl, test_data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


0.865